<a href="https://colab.research.google.com/github/Wejdan999/T55/blob/main/Training_week3_day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier,AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_wine
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Load the dataset


In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation',
           'relationship', 'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']
data = pd.read_csv(url, header=None, names=columns, na_values=' ?')


# Preprocess the data (if necessary)

In [ ]:
data = data.dropna()

data = pd.get_dummies(data, drop_first=True)


X = data.drop('income_ >50K', axis=1)
y = data['income_ >50K']



# Split the Dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Initialize and Train the Classifiers

## Random Forest
Initialize and train a Random Forest classifier.

In [ ]:
random_forest_classifier = RandomForestClassifier(n_estimators=50, random_state=42)
random_forest_classifier.fit(X_train, y_train)


In [ ]:
predictions = random_forest_classifier.predict(X_test)

### Evaluate the model performance

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print(f'Random Forest Model Accuracy: {accuracy * 100:.2f}%')

## Bagging Meta-estimator
Initialize a K-Nearest Neighbors classifier and use it as the base estimator for the Bagging classifier.

In [ ]:
base_estimator = KNeighborsClassifier()
bagging_classifier = BaggingClassifier(base_estimator, n_estimators=50, random_state=42)

bagging_classifier.fit(X_train, y_train)



In [ ]:
predictions = bagging_classifier.predict(X_test)

### Evaluate the model performance

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print(f'Bagging Classifier Model Accuracy: {accuracy * 100:.2f}%')

## Pasting
Initialize a Decision Tree classifier and use it as the base estimator for a Bagging classifier with Pasting (without replacement).

In [ ]:
base_estimator = DecisionTreeClassifier()
pasting_classifier = BaggingClassifier(base_estimator, n_estimators=50, max_samples=0.7, bootstrap=False, random_state=42)

pasting_classifier.fit(X_train, y_train)

In [ ]:
predictions = pasting_classifier.predict(X_test)

### Evaluate the model performance

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print(f'Pasting Classifier Model Accuracy: {accuracy * 100:.2f}%')

## Roughly Balanced Bagging (RBB)
Implement Roughly Balanced Bagging by manually creating balanced bootstrap samples and aggregating predictions from multiple Decision Tree classifiers.

In [ ]:
n_estimators = 100

ensemble_preds = np.zeros((n_estimators, len(X_test)))
ensemble_models = []

for i in range(n_estimators):

    pos_indices = np.where(y_train == 1)[0]
    neg_indices = np.where(y_train == 0)[0]

    chosen_pos_indices = np.random.choice(pos_indices, size=len(pos_indices), replace=True)
    chosen_neg_indices = np.random.choice(neg_indices, size=len(pos_indices), replace=True)

    balanced_sample_indices = np.concatenate([chosen_pos_indices, chosen_neg_indices])
    np.random.shuffle(balanced_sample_indices)

    X_train_balanced = X_train.iloc[balanced_sample_indices]
    y_train_balanced = y_train.iloc[balanced_sample_indices]


    tree_clf = DecisionTreeClassifier(random_state=i)
    tree_clf.fit(X_train_balanced, y_train_balanced)
    ensemble_models.append(tree_clf)





In [ ]:
ensemble_preds[i] = tree_clf.predict(X_test)

In [ ]:
final_preds = np.round(np.mean(ensemble_preds, axis=0))

### Evaluate the model performance

In [ ]:
accuracy = accuracy_score(y_test, final_preds)
print(f'Roughly Balanced Bagging Model Accuracy: {accuracy:.2f}')


### Step 2: Load and Preprocess the Dataset
Load the dataset and preprocess it. This includes handling missing values, encoding categorical variables, and splitting the data into features and target variables.

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip"
!wget $url
!unzip bank-additional.zip
data = pd.read_csv('bank-additional/bank-additional-full.csv', delimiter=';') # Load the data
data = pd.get_dummies(data, drop_first=True)

X = data.drop('y_yes', axis=1)
y = data['y_yes']

### Step 3: Split the Dataset
Split the dataset into training and testing sets to evaluate the performance of the models.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

### Step 4: Initialize and Train the AdaBoost Classifier
Initialize a Decision Tree classifier and use it as the base estimator for the AdaBoost classifier.

In [ ]:
ase_estimator = DecisionTreeClassifier(max_depth=1)
adaboost_classifier = AdaBoostClassifier(base_estimator=base_estimator, n_estimators=50, random_state=42)


adaboost_classifier.fit(X_train, y_train)

In [ ]:
predictions = adaboost_classifier.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f'AdaBoost Classifier Model Accuracy: {accuracy * 100:.2f}%')

## XGBoost Tutorial


### Step 3: Split the Dataset
Split the dataset into training and testing sets to evaluate the performance of the models.

In [ ]:

data = pd.get_dummies(data, drop_first=True)

X = data.drop('y_yes', axis=1)
y = data['y_yes']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Step 4: Initialize and Train the XGBoost Classifier
Initialize and train the XGBoost classifier.

In [ ]:
xgb_classifier = XGBClassifier(n_estimators=50, random_state=42)
xgb_classifier.fit(X_train, y_train)


In [ ]:
predictions = xgb_classifier.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f'XGBoost Classifier Model Accuracy: {accuracy * 100:.2f}%')

## Gradient Boosting Tutorial


###  Initialize and Train the Gradient Boosting Classifier
Initialize and train the Gradient Boosting classifier.

In [ ]:
gradient_boosting_classifier = GradientBoostingClassifier(n_estimators=50, random_state=42)
gradient_boosting_classifier.fit(X_train, y_train)


In [ ]:
predictions = gradient_boosting_classifier.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f'Gradient Boosting Classifier Model Accuracy: {accuracy * 100:.2f}%')

### Load the dataset

In [ ]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
data = pd.read_csv(url, names=columns)

### Preprocess the data (if necessary)

In [ ]:
for col in ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']:
    data[col] = data[col].replace(0, data[col].mean())

label_encoder = LabelEncoder()
data['Outcome'] = label_encoder.fit_transform(data['Outcome'])


X = data.drop('Outcome', axis=1)
y = data['Outcome']

scaler = StandardScaler()
X = scaler.fit_transform(X)

### Split the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=12)

### Implement a stacking model

In [ ]:
base_models = [
    ('decision_tree', DecisionTreeClassifier(random_state=42)),
    ('svc', SVC(probability=True, random_state=42)),
    ('knn', KNeighborsClassifier()),
    ('random_forest', RandomForestClassifier(random_state=42))
]

meta_model = LogisticRegression()

In [ ]:
stacking_classifier = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)
stacking_classifier.fit(X_train, y_train)

In [ ]:
predictions = stacking_classifier.predict(X_test)

### Evaluate the model performance

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print(f'Stacking Classifier Model Accuracy: {accuracy * 100:.2f}%')